# Data processing

In [1]:
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv(r'store-sales-time-series-forecasting\train.csv')
holiday_events = pd.read_csv(r'store-sales-time-series-forecasting\holidays_events.csv')
stores = pd.read_csv(r'store-sales-time-series-forecasting\stores.csv')
test = pd.read_csv(r'store-sales-time-series-forecasting\test.csv')
oil = pd.read_csv(r'store-sales-time-series-forecasting\oil.csv')

In [3]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [4]:
def process_data(df,stores,holiday_events,oil):
    # --- deal store data
    merged_df = pd.merge(df, stores, on="store_nbr", how="left")
    merged_df = merged_df.drop(columns=['type','cluster','id'])
    # --- deal holiday data
    holiday_df = holiday_events[holiday_events['transferred']==False]
    holiday_df['is_holiday'] = 1
    holiday_df = holiday_df[['date','is_holiday']].drop_duplicates(subset='date')
    merged_df = pd.merge( merged_df, holiday_df, on = "date", how = "left")
    merged_df['is_holiday'] = merged_df['is_holiday'].fillna(0).astype(int)
    # --- deal oil data
    merged_df = pd.merge( merged_df, oil, on = "date", how = "left")
    merged_df['dcoilwtico'] = merged_df['dcoilwtico'].fillna(0)
    # --- add day and weekend key
    merged_df['date'] = pd.to_datetime(merged_df['date'])
    merged_df['day'] = merged_df["date"].dt.day
    merged_df['month'] = merged_df["date"].dt.month
    merged_df['year'] = merged_df["date"].dt.year
    merged_df['is_weekend'] = merged_df['date'].apply(lambda x : 1 if x.dayofweek>=5 else 0)
    return merged_df
merged_train = process_data(train,stores,holiday_events,oil)
merged_test = process_data(test,stores,holiday_events,oil)

In [5]:
merged_train

,date,store_nbr,family,sales,onpromotion,city,state,is_holiday,dcoilwtico,day,month,year,is_weekend
0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,1,0.00,1,1,2013,0
1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,1,0.00,1,1,2013,0
2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,1,0.00,1,1,2013,0
3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,1,0.00,1,1,2013,0
4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,1,0.00,1,1,2013,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,1,47.57,15,8,2017,0
3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,1,47.57,15,8,2017,0
3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,1,47.57,15,8,2017,0
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,1,47.57,15,8,2017,0


In [6]:
import sklearn
from sklearn.preprocessing import LabelEncoder,StandardScaler
def encode_scale(df, encode_col, scale_col):
    encode = LabelEncoder()
    scaler = StandardScaler()
    for col in encode_col:
        df['encode_'+col] = encode.fit_transform(df[col])
    for col in scale_col:
        df['scale_'+col] = scaler.fit_transform(df[[col]])
    return df,encode,scaler

In [7]:
encode_col = ['family']
scale_col = ['onpromotion','dcoilwtico']
pro_df,encode,scaler = encode_scale(merged_train, encode_col, scale_col)
pro_df = pro_df.drop(columns=['date','family','onpromotion','city','state','dcoilwtico'])

In [8]:
feature_col = ['month','day','store_nbr', 'is_holiday', 'is_weekend', 'encode_family', 'scale_dcoilwtico', 'scale_onpromotion']
pro_df[feature_col]

,month,day,store_nbr,is_holiday,is_weekend,encode_family,scale_dcoilwtico,scale_onpromotion
0,1,1,1,1,0,0,-1.235870,-0.213012
1,1,1,1,1,0,1,-1.235870,-0.213012
2,1,1,1,1,0,2,-1.235870,-0.213012
3,1,1,1,1,0,3,-1.235870,-0.213012
4,1,1,1,1,0,4,-1.235870,-0.213012
...,...,...,...,...,...,...,...,...
3000883,8,15,9,1,0,28,0.017372,-0.213012
3000884,8,15,9,1,0,29,0.017372,-0.131172
3000885,8,15,9,1,0,30,0.017372,11.899391
3000886,8,15,9,1,0,31,0.017372,0.441712


In [9]:
from sklearn.model_selection import train_test_split
x = pro_df[feature_col]
y = pro_df['sales']
print("Length of x:", len(x))
print("Length of y:", len(y))

# 確保數據一致無遺漏問題
if len(x) == len(y):
    # 分割數據集
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42)
    
    # 確認分割後形狀
    print("x_train shape:", x_train.shape)
    print("y_train shape:", y_train.shape)

Length of x: 3000888
Length of y: 3000888
x_train shape: (2400710, 8)
y_train shape: (2400710,)


# LinearRegression

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# x_train,y_train,x_test,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

model = LinearRegression()

model.fit(x_train,y_train)

y_pred = model.predict(x_test)

mse = mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)
# 顯示結果
print("Coefficients:", model.coef_)  # 特徵的係數
print("Intercept:", model.intercept_)  # 截距
print("Mean Squared Error (MSE):", mse)  # 均方誤差
print("R-squared (R2):", r2)  # R^2 分數

Coefficients: [  2.71220731  -1.56497878   2.67027139  13.79099891 157.97924979
 -10.94361805   1.15827077 466.83718912]
Intercept: 419.4791697748181
Mean Squared Error (MSE): 997361.341920931
R-squared (R2): 0.19749610894736358


In [12]:
encode_col = ['family','city','state']
scale_col = ['onpromotion']
test_df,_,_ = encode_scale(merged_test, encode_col, scale_col)
test_df = test_df.drop(columns=['date','family','onpromotion','city','state'])
test_df['scale_sales'] = model.predict(test_df)
test_df['sales'] = scaler.inverse_transform(test_df[['scale_sales']])[:, 0]

In [19]:
combine_df = pd.concat([ test_df.reset_index(drop=True), test.reset_index(drop=True)], axis = 1)[['id','sales']]

In [21]:
combine_df.to_csv('submission_regression.csv', index=False)

# Random Forest Regressor

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 建立並配置 Random Forest Regressor
rf_model = RandomForestRegressor(
    n_estimators=500,       # 樹的數量（可以增加以提升擬合能力，但可能增加計算成本）
    max_depth=31,           # 最大樹深度（限制以避免過擬合）
    random_state=42,        # 固定隨機狀態保證結果穩定
    min_samples_split=10,   # 分裂的最小樣本數
    min_samples_leaf=5,     # 葉節點的最小樣本數
    n_jobs=-1               # 使用所有處理器並行運算
)
rf_model.fit(x_train,y_train)

y_pred_rf = rf_model.predict(x_test)

mse = mean_squared_error(y_test,y_pred_rf)
r2 = r2_score(y_test,y_pred_rf)

print("Mean Squared Error (MSE):", mse)  # 均方誤差
print("R-squared (R2):", r2)  # R^2 分數

Mean Squared Error (MSE): 110455.36576760026
R-squared (R2): 0.9111246274640865


In [20]:
encode_col = ['family']
scale_col = ['onpromotion','dcoilwtico']
test_df,_,_ = encode_scale(merged_test, encode_col, scale_col)
test_df = test_df.drop(columns=['date','family','onpromotion','city','state','dcoilwtico'])[feature_col]
test_df['sales'] = rf_model.predict(test_df)
combine_df_rf = pd.concat([ test_df.reset_index(drop=True), test.reset_index(drop=True)], axis = 1)[['id','sales']]
combine_df_rf['sales'] = combine_df_rf['sales'].apply(lambda x: 0 if x < 0 else x)
combine_df_rf.to_csv('submission_randomforest.csv', index=False)

# Gradient Boosting Regressor

In [10]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 建立並配置 Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(
    n_estimators=200,            # 樹的數量（增大可能提升效果，但需注意計算成本）
    learning_rate=0.05,           # 學習率（減少可能減低步長，避免過擬合）
    max_depth=5,                 # 樹的深度（增加過擬合可能性）
    subsample=0.8,               # 每棵樹所使用的子樣本比例（控制過擬合）
    random_state=42              # 保持結果穩定
)

# 訓練模型
gb_model.fit(x_train, y_train)

# 預測
y_pred_gb = gb_model.predict(x_test)

# 評估模型
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

# 顯示結果
print("Gradient Boosting Regressor Results:")
print("Mean Squared Error (MSE):", mse_gb)
print("R-squared (R2):", r2_gb)

Gradient Boosting Regressor Results:
Mean Squared Error (MSE): 242918.62471346091
R-squared (R2): 0.8045411092771608


In [13]:
encode_col = ['family','city','state']
scale_col = ['onpromotion']
test_df,_,_ = encode_scale(merged_test, encode_col, scale_col)
test_df = test_df.drop(columns=['date','family','onpromotion','city','state'])
test_df['scale_sales'] = gb_model.predict(test_df)
test_df['sales'] = scaler.inverse_transform(test_df[['scale_sales']])[:, 0]
combine_df_rf = pd.concat([ test_df.reset_index(drop=True), test.reset_index(drop=True)], axis = 1)[['id','sales']]
combine_df_rf.to_csv('submission.csv', index=False)

# LGBMRegressor

In [11]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score


# 建立並配置 LGBM Regressor
lgb_model = lgb.LGBMRegressor(
    n_estimators=500,             # 樹的數量
    learning_rate=0.05,           # 學習率
    num_leaves=31,                # 每棵樹葉節點的最大數量
    max_depth=5,                  # 樹的深度
    subsample=0.8,                # 每棵樹使用子樣本比例
    random_state=42               # 保持結果穩定
)

# 訓練模型
lgb_model.fit(x_train, y_train)

# 預測
y_pred_lgb = lgb_model.predict(x_test)

# 評估模型
mse_lgb = mean_squared_error(y_test, y_pred_lgb)
r2_lgb = r2_score(y_test, y_pred_lgb)

# 顯示結果
print("LGBM Regressor Results:")
print("Mean Squared Error (MSE):", mse_lgb)
print("R-squared (R2):", r2_lgb)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.053628 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 2400710, number of used features: 8
[LightGBM] [Info] Start training from score 357.301279
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [17]:
encode_col = ['family']
scale_col = ['onpromotion','dcoilwtico']
test_df,_,_ = encode_scale(merged_test, encode_col, scale_col)
test_df = test_df.drop(columns=['date','family','onpromotion','city','state','dcoilwtico'])[feature_col]
test_df['sales'] = lgb_model.predict(test_df)
combine_df_rf = pd.concat([ test_df.reset_index(drop=True), test.reset_index(drop=True)], axis = 1)[['id','sales']]
combine_df_rf['sales'] = combine_df_rf['sales'].apply(lambda x: 0 if x < 0 else x)
combine_df_rf.to_csv('submission.csv', index=False)